# Projeto de Sinais

## Sinais e Sistemas Dinâmicos - Prof. Derzu Omaia

## Jansepetrus Brasileiro Pereira e Nathália de Vasconcelos Silva

Descrição do projeto: https://www.dropbox.com/s/wzibx7g7136um4z/Projeto.pdf?dl=0

### Importar todas as bibliotecas necessárias para o projeto

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.fftpack import fft
from scipy.signal import butter, lfilter

### Definicao das funções de usuário a serem utilizadas

In [ ]:
def plot_wav(wav, sample_rate, save_plot=False, nome_plot="", sufixo_nome=""):
    if(nome_plot == ""):
        from datetime import datetime
        nome_plot = str(datetime.now()).replace(":","-")
        
    times = np.arange(len(wav))/float(sample_rate)

    plt.figure(figsize=(30, 4))

    if(wav.ndim == 2):
        plt.fill_between(times, wav[:,0], wav[:,1], color='k') # para dual-channel
    else:
        plt.fill_between(times, wav)

    plt.xlim(times[0], times[-1])
    plt.xlabel('time (s)')
    plt.ylabel('amplitude')
    
    if(save_plot):
        plt.savefig(nome_plot + "_" + sufixo_nome + '.png', dpi=210)
    plt.show()

def plot_wav_freqDomain(fft_wav, freq, N, save_plot=False, nome_plot="", sufixo_nome=""):
    if(nome_plot == ""):
        from datetime import datetime
        nome_plot = str(datetime.now()).replace(":","-")
        
    plt.plot(freq, 1.0/N * np.abs(fft_wav))
    plt.grid()
    if(save_plot):
        plt.savefig(nome_plot + "_" + sufixo_nome + '.png', dpi=210)
    plt.show()

### Definição dos Parâmetros de Execução

In [ ]:
####
# Parametros de Execucao
####
PLOT_SINAL_ORIGINAL = True
PLOT_SINAL_FFT = True
PLOT_SINAL_FFT_SHIFT = True
PLOT_SINAL_FFT_SHIFT_BUTTER = True
MONO = True
MONO_ESQ = True
STEREO = not MONO
MONO_DIR = not MONO_ESQ

### Percorrer a pasta do dataset IRMAS, abrir todos os arquivos .wav e aplicar a transformada de Fourier

In [ ]:
ROOT_PATH = 'IRMAS-TrainingData_red/'

signal_dataset = []

for root, dirs, files in os.walk(ROOT_PATH):  
    for filename in files:
        ####
        # Leitura do arquivo WAV
        ####
        sample_rate, wav_data = wavfile.read(root + '\\' + filename)

        ####
        # Captura do sinal em um ndarray
        ####
        if(wav_data.ndim == 2):
            if(STEREO):
                x = wav_data
        if(MONO_DIR):
            x = wav_data[0:,1] # canal da direita
        if(MONO_ESQ):
            x = wav_data[0:,0] # canal da esquerda
        
        if (PLOT_SINAL_ORIGINAL):
            plot_wav(x, sample_rate, sufixo_nome="ORIG", save_plot=True, nome_plot=filename)

        ####
        # Especificacoes do dominio do tempo
        ####
        L = x.size
        dt = 1/sample_rate # Periodo de amostragem
        t = np.arange(0, (L/sample_rate)-dt, dt) # Discretizacao da amostragem ao passo ((L/sample_rate)-dt)
        N = t.size +1
        
        ####
        # Aplicacao da FFT e Calculo da Frequencia do sinal
        ####
        fft_output = fft(x) # Fourier.
        fft_out_freq = np.fft.fftfreq(N,dt)
        
        if (PLOT_SINAL_FFT):
            plot_wav_freqDomain(fft_output, fft_out_freq, N, sufixo_nome="FFT", save_plot=True, nome_plot=filename)
        
        freq_max = np.amax(fft_out_freq)
        
        ####
        # Movimentacao dos componentes de frequencia zero para o centro do espectro
        ####
        fft_out_freq = np.fft.fftshift(fft_out_freq)
        fft_shifted = np.fft.fftshift( fft_output ) # Desloca o centro zero da transformada para o centro.
        
        if (PLOT_SINAL_FFT_SHIFT):
            plot_wav_freqDomain(fft_shifted, fft_out_freq, N, sufixo_nome="FFT-SHIFT", save_plot=True, nome_plot=filename)
        
        ####
        # Aplicacao do Filtro Passa-Baixa Butterworth
        ####
        freq_corte = 5000 # TODO: Precisa descobrir se essa e' uma boa frequencia de corte para utilizar no Passa-Baixa Butterworth
        b, a = butter(6, freq_corte/(0.5 * sample_rate), btype='low') # 1st order Butterworth low-pass
        fft_X_filtro = lfilter(b, a, fft_shifted)
        
        if(PLOT_SINAL_FFT_SHIFT_BUTTER):
            plot_wav_freqDomain(fft_X_filtro, fft_out_freq, N, sufixo_nome="FFT-SHIFT-FILT", save_plot=True, nome_plot=filename)
        
        ####
        # Adicao do sinal filtrado ao DataSet, para ser transformado em um DataFrame.
        ####
        signal_dataset.append(fft_X_filtro)

df_data = pd.DataFrame(signal_dataset)

### Guardar o DataFrame no PC, para evitar executar o código acima

In [ ]:
DATAFRAME_NAME = 'WAV_DF.pkl'
df_data.to_pickle(DATAFRAME_NAME)

### Ler um DataFrame salvo na máquina

In [ ]:
DATAFRAME_NAME = 'WAV_DF.pkl'
data_frame = pd.read_pickle(DATAFRAME_NAME)

#print(data_frame)
#data_frame.describe()